### Install dependencies

In [1]:
# Change into ydb-sqlalchemy from pypi after release
%pip install --force-reinstall /Users/ovcharuk/work/ydb-sqlalchemy/dist/ydb_sqlalchemy-0.0.1b23-py2.py3-none-any.whl

Processing /Users/ovcharuk/work/ydb-sqlalchemy/dist/ydb_sqlalchemy-0.0.1b23-py2.py3-none-any.whl
  Using cached SQLAlchemy-2.0.36-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.7 kB)
  Using cached ydb-3.18.8-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached ydb_dbapi-0.0.1b8-py3-none-any.whl.metadata (1.7 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached grpcio-1.67.1-cp39-cp39-macosx_10_9_universal2.whl.metadata (3.9 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached aiohttp-3.10.10-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.5.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cac

In [2]:
%pip install pandas jupysql matplotlib


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Actions with YDB

### Load DataFrame to work with

In [3]:
import pandas as pd
df = pd.read_csv('/Users/ovcharuk/work/ydb-python-examples/Mall_Customers.csv')
df

,id,sex,age,income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


### Create SA Engine and connect

In [4]:
import sqlalchemy as sa
engine = sa.create_engine("yql+ydb://localhost:2136/local")

In [5]:
conn = engine.connect()

### Prepare database to load dataframe into
There's no way to specify PK directly from `df.to_sql`, and `YDB` does not support tables without PK, so we have to prepare database first

In [6]:
conn.execute(sa.text("DROP TABLE IF EXISTS mall_customers"))
conn.execute(sa.text(
    """
    CREATE TABLE mall_customers (
	`id` Int64, 
	`sex` UTF8, 
	`age` Int64, 
	`income` Int64, 
	`spending_score` Int64,
    primary key (id)
    )
    """
))

### Load dataframe into YDB

In [7]:
df.to_sql("mall_customers", conn, if_exists='append', index=False)

-1

### Read YDB table into DataFrame

In [8]:
new_df = pd.read_sql("SELECT * FROM mall_customers", conn)

In [9]:
new_df

,age,id,income,sex,spending_score
0,19,1,15,Male,39
1,21,2,15,Male,81
2,20,3,16,Female,6
3,23,4,16,Female,77
4,31,5,17,Female,40
...,...,...,...,...,...
195,35,196,120,Female,79
196,45,197,126,Female,28
197,32,198,126,Male,74
198,32,199,137,Male,18


### Using `sql` magic to work with YDB

In [10]:
%load_ext sql

In [11]:
%sql yql+ydb://localhost:2136/local

Connecting to 'yql+ydb://localhost:2136/local'

In [12]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
# https://community.deeplearning.ai/t/error-while-querying-db-from-jupyter-notebook/716351

In [13]:
%%sql
SELECT sex, spending_score 
    FROM mall_customers
    WHERE age > 30

Running query in 'yql+ydb://localhost:2136/local'

138 rows affected.

sex,spending_score
Female,40
Female,6
Male,3
Male,14
Female,99
Female,15
Male,13
Female,35
Male,29
Female,98
